In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file
from bokeh.layouts import column, row
from bokeh.models import OpenURL, TapTool, CustomJS, Div, Callback
from bokeh import events

output_notebook()

Loading BokehJS ...

In [2]:
from dandi.dandiapi import DandiAPIClient
import json
import pandas as pd
import numpy as np
from urllib.parse import quote, unquote

In [3]:
def get_data(sub):
    api = DandiAPIClient("https://api.dandiarchive.org/api")
    ds = api.get_dandiset("000108")

    assets = list(ds.get_assets_by_glob(f"*{sub}/*SPIM.ome.zarr"))
    photos = list(ds.get_assets_by_glob(f"*{sub}/*_photo.jpg"))
    df = pd.DataFrame([dict([val.split("-")
                             for val in asset.path.split("/")[-1].split(".")[0].split("_")
                             if "-" in val]) for asset in assets])
    samples = sorted(df['sample'].unique(), key=lambda x: int(x.split("R")[0]))
    samples_w_sessions = sorted([val[0] for val in df.groupby(['sample', 'ses'])], key=lambda x: int(x[0].split("R")[0]))
    return df, ds, samples, photos

In [4]:
def get_photo_url(photos, sample):
    photo_url = [photo.get_content_url(regex='s3') for photo in photos if f'_sample-{sample}_' in photo.path]
    return photo_url[0] if photo_url else None

In [5]:
sub = 'sub-MITU01'
# sub = 'sub-MITU01h3'
df, ds, samples, photos = get_data(sub)
df['sample'] = df['sample'].apply(lambda x: int(x.split('R')[0]) if 'R' in x else int(x))

In [6]:
df.head()

,sub,ses,sample,stain,run,chunk
0,MITU01,20210720h20m19s32,127,YO,1,8
1,MITU01,20210720h20m19s32,127,YO,1,9
2,MITU01,20210721h22m29s00,128,LEC,1,10
3,MITU01,20210521h17m17s06,178,NN,1,3
4,MITU01,20210521h17m17s06,178,NN,1,1


In [7]:
stains = df['stain'].unique().tolist()
print(stains)
remap = dict(calretinin='CR', npy='NPY')

['YO', 'LEC', 'NN', 'NPY', 'CR', 'calretinin', 'npy']


In [8]:
df.stain = df.stain.apply(lambda x: remap[x] if x in remap else x)
stains = df['stain'].unique().tolist()
print(stains)

['YO', 'LEC', 'NN', 'NPY', 'CR']


In [9]:
samples = range(df['sample'].min(), df['sample'].max() + 1)
samples

range(6, 180)

In [10]:
colormap = ["#444444", "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99",
            "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]

In [11]:
df.head()

,sub,ses,sample,stain,run,chunk
0,MITU01,20210720h20m19s32,127,YO,1,8
1,MITU01,20210720h20m19s32,127,YO,1,9
2,MITU01,20210721h22m29s00,128,LEC,1,10
3,MITU01,20210521h17m17s06,178,NN,1,3
4,MITU01,20210521h17m17s06,178,NN,1,1


In [12]:
mi_chunk = df.groupby(['sample', 'stain', 'ses']).chunk.count()

In [13]:
dfcat = pd.concat((mi_chunk.index.to_frame(), mi_chunk), axis=1)
#dfcat = dfcat.head(10)
dfcat

sample stain                ses  chunk
sample stain ses                                                      
6      LEC   20220326h16m14s39       6   LEC  20220326h16m14s39      4
       NN    20220326h16m14s39       6    NN  20220326h16m14s39      4
       YO    20220326h16m14s39       6    YO  20220326h16m14s39      4
8      LEC   20220326h07m16s28       8   LEC  20220326h07m16s28      4
       NN    20220326h07m16s28       8    NN  20220326h07m16s28      4
...                                ...   ...                ...    ...
178    NN    20210521h17m17s06     178    NN  20210521h17m17s06      3
       NPY   20210521h17m17s06     178   NPY  20210521h17m17s06      3
179    LEC   20210524h18m29s31     179   LEC  20210524h18m29s31      3
       NPY   20210524h18m29s31     179   NPY  20210524h18m29s31      3
       YO    20210524h18m29s31     179    YO  20210524h18m29s31      3

[343 rows x 4 columns]

In [14]:
cubehelix_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.r = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

cubehelix2_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.g = -0.14861 * cosangle + 1.78277 * sinangle;
    result.r = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

cubehelix3_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.b = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.r = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

cubehelix4_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.r = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

cubehelix5_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.g = -0.14861 * cosangle + 1.78277 * sinangle;
    result.b = -0.29227 * cosangle + -0.90649 * sinangle;
    result.r = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

cubehelix6_template = """
#uicontrol float brightness slider(min=0.0, max=100.0, default=%f)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.b = -0.14861 * cosangle + 1.78277 * sinangle;
    result.r = -0.29227 * cosangle + -0.90649 * sinangle;
    result.g = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

ng_colormap = {"LEC": cubehelix_template % 50,
            "YO": cubehelix2_template % 50,
            "NN": cubehelix3_template % 50,
            "CR": cubehelix4_template % 50,
            'NPY': cubehelix5_template % 50,
            'IBA1': cubehelix6_template % 50,
            'SST': cubehelix4_template % 50}

def get_url(ds, subj, sample, stains):
    layers = []
    for stain in stains:
        zarrs = list(ds.get_assets_by_glob(f"*{subj}/*_sample-{sample}_stain-{stain}_run-1*.ome.zarr"))

        sources = [f"zarr://{val.get_content_url(regex='s3')}"
                  for val in sorted(zarrs, key=lambda x: int(x.path.split("_chunk-")[1].split("_")[0]))]
        # print([val.path for val in sorted(zarrs, key=lambda x: int(x.path.split("_chunk-")[1].split("_")[0]))])
        if len(zarrs):
            val = zarrs[0]
            layer = dict(
                source=sources,
                type="image",
                shader=ng_colormap[stain],
                name=val.path.split("_sample-")[1].split("_")[0] + f'-{stain}' + "-" + f'{len(zarrs)}',
                tab='rendering',
            )
            layers.append(layer)

    ng_url = "https://neuroglancer-demo.appspot.com/"
    ng_str = json.dumps(dict(dimensions={"t":[1,"s"],
                                         "z":[0.000002285,"m"],
                                         "y":[0.0000032309999999999996,"m"],
                                         "x":[0.000002285,"m"]},
                             displayDimensions=["z","y","x"],
                             crossSectionScale=50,
                             projectionScale=500000,
                             layers=layers,
                             showDefaultAnnotations=False,
                             layerListPanel={'visible': len(layers)>1},
                             layout="yz"))
    url = f"{ng_url}#!%s" % quote(ng_str)
    return url

In [15]:
data=dict(
    samples=[str(sample) for sample in dfcat['sample'].tolist()],
    stains=dfcat['stain'].tolist(),
    colors=[colormap[stains.index(stain)] for stain in dfcat['stain'].tolist()],
    sessions=dfcat['ses'].tolist(),
    chunks=dfcat['chunk'].tolist(),
    url=[unquote(get_url(ds, sub, val[1]["sample"], [val[1]["stain"]])) for val in dfcat.iterrows()],
    photo=[get_photo_url(photos, val[1]["sample"]) for val in dfcat.iterrows()]
)

In [16]:
output_file(f"layout-{sub}.html")
plots = []
ncols = 60
samples = range(dfcat['sample'].min(), dfcat['sample'].max() + 1)
numpages = int(np.ceil(max(samples)/ncols))

for i in range(1, numpages + 1):
    if (max(samples) - ncols * (i-1))/ncols < 0.8:
        ncols = min(max(samples) - ncols * (i-1), ncols)
    p = figure(title=f"Sample coverage: {[ncols*(i - 1) + 1, ncols*i]}",
               x_axis_location="below", tools="hover,save,tap",
               x_range=[str(val) for val in range(ncols*(i - 1) + 1, ncols*i + 1)], y_range=stains,
               tooltips = [('sample', '@samples'),
                           ('chunks', '@chunks'), 
                           ('sessions', '@sessions')])
    p.width = max(15*ncols, 200)
    p.height = 125
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = np.pi/3
    p.output_backend = "svg"

    p.rect('samples', 'stains', 0.9, 0.9, source=data,
           color='colors', line_color=None,
           hover_line_color='red') #, hover_color='red') #colors')
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url="@url")
    plots.append(p)
show(column(plots))